In [97]:
%run input/Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [98]:
iOption = 0
rebin = 2
pt_low, pt_high = 1, -1
phi_v_low, phi_v_high = 2, 31

In [99]:
isigma = 2
N_centr =5
N_options = 4
centralities = ["0-88%","0-20%","20-40%","40-60%","60-88%"]
centrality = [0,4,0,0,1,1,2,2,3,4]
colors=[1,2,4,6,root.kGreen+2]
file_path="input/"
file_name="m_ee_Run14AuAU_first_test_12runs.root"#"HeAu_19018_Artem_newcuts_E04.root"#HeAu_18968_Artem_newcuts

In [100]:
hists_ep_FG_read, hists_ep_BG_read, hists_ee_FG_read, hists_ee_BG_read = [], [], [], []
hist_stat_read=[]
infile = root.TFile.Open(file_path+file_name, "read")

for iOpt in range(1,N_options+1):
    histo_name = f"inv_m_ee_V{iOpt}_"
    histo_name_like = f"phi_ee_V{iOpt}_"
    hists_ep_FG_read_centr, hists_ep_BG_read_centr, hists_ee_FG_read_centr, hists_ee_BG_read_centr = [], [], [], []
    hist_stat_centr = []
    for icent in range(N_centr):
        hist_FG = infile.Get("c00_z00_r00/"+histo_name+"FG12")
        hist_FG.Add(hist_FG,-1)
        hist_FG.SetDirectory(root.nullptr)
        hist_BG = infile.Get("c00_z00_r00/"+histo_name+"BG12")
        hist_BG.Add(hist_BG,-1)
        hist_BG.SetDirectory(root.nullptr)
        hist_FG_like = infile.Get("c00_z00_r00/"+histo_name_like+"FG11")
        hist_FG_like.Add(hist_FG,-1)
        hist_FG_like.SetDirectory(root.nullptr)
        hist_BG_like = infile.Get("c00_z00_r00/"+histo_name_like+"BG11")
        hist_BG_like.Add(hist_BG,-1)
        hist_BG_like.SetDirectory(root.nullptr)
        hist_stat = infile.Get("c00_z00_r00/PoolStatistics")
        hist_stat.Add(hist_stat,-1)
        hist_stat.SetDirectory(root.nullptr)
        for i in range(centrality[icent*2],centrality[icent*2+1]+1):
            for j in range(2):
                hist_FG.Add(infile.Get(f"c0{i}_z0{j}_r00/"+histo_name+"FG12"))
                hist_BG.Add(infile.Get(f"c0{i}_z0{j}_r00/"+histo_name+"BG12"))
                hist_FG_like.Add(infile.Get(f"c0{i}_z0{j}_r00/"+histo_name_like+"FG11"))
                hist_BG_like.Add(infile.Get(f"c0{i}_z0{j}_r00/"+histo_name_like+"BG11"))
                hist_FG_like.Add(infile.Get(f"c0{i}_z0{j}_r00/"+histo_name_like+"FG22"))
                hist_BG_like.Add(infile.Get(f"c0{i}_z0{j}_r00/"+histo_name_like+"BG22"))
                hist_stat.Add(infile.Get(f"c0{i}_z0{j}_r00/PoolStatistics"))
        hists_ep_FG_read_centr.append(hist_FG)
        hists_ep_BG_read_centr.append(hist_BG)
        hists_ee_FG_read_centr.append(hist_FG_like)
        hists_ee_BG_read_centr.append(hist_BG_like)
        hist_stat_centr.append(hist_stat)
    hists_ep_FG_read.append(hists_ep_FG_read_centr)
    hists_ep_BG_read.append(hists_ep_BG_read_centr)
    hists_ee_FG_read.append(hists_ee_FG_read_centr)
    hists_ee_BG_read.append(hists_ee_BG_read_centr)
    hist_stat_read.append(hist_stat_centr)


infile.Close()

In [101]:
def emc_fitter(InHistFG, InHistBG, centr, seed=1, sigma=2, fits_mean=0, fits_sigma=0):

    hist_pt_bin = InHistFG[icent].ProjectionY("kek")

    eta_FG_porj = InHistFG[centr].ProjectionX(f"FG{seed+10*centr}",pt_low,pt_high,phi_v_low,phi_v_high)
    eta_BG_porj = InHistBG[centr].ProjectionX(f"BG{seed+10*centr}",pt_low,pt_high,phi_v_low,phi_v_high)

    eta_FG_porj.Rebin(2)
    eta_BG_porj.Rebin(2)

    left = eta_FG_porj.FindBin(1.)
    right = eta_FG_porj.FindBin(2.)

    scale = eta_FG_porj.Integral(left,right)/(eta_BG_porj.Integral(left,right)+0.1)

    return scale


In [102]:
def emc_centr_fitter(InHistFG, InHistBG, centralities, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    hist_ser = []
    for centr in centralities:
        hist_ser.append(emc_fitter(InHistFG, InHistBG, centr, seed, sigma, fits_mean, fits_sigma))
    return hist_ser

In [103]:
import multiprocess as mp

Ntr = N_options

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_ee_FG_read[i], hists_ee_BG_read[i], [0,1,2,3,4], i, isigma) for i in range(Ntr)])
pool.close()

In [104]:
c3 = root.TCanvas(f"c2",f"c2",500*2,500*2)
c3.Divide(2,2)
legends =[]
for icentr in range(N_centr):
    c3.cd(icentr+1)
    proj = hists_ee_FG_read[iOption][icentr].ProjectionY(f"px{icentr}{ihist}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj_BG = hists_ee_BG_read[iOption][icentr].ProjectionY(f"px_BG_{icentr}{ihist}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj_BG.Scale(output_array[iOption][icent])
    proj.Rebin(rebin)
    proj_BG.Rebin(rebin)
    Format_Hist_total(proj,"#Delta#phi","dN/d#phi",0.22,0.15,0.03,0.1)
    proj.GetYaxis().SetTitleOffset(1.5)
    proj.Draw("")
    proj.SetMinimum(0)
    proj.SetLineWidth(3)
    proj_BG.SetLineWidth(3)
    proj_BG.SetLineColor(2)
    proj_BG.Draw("same")
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()
        
    

c3.Draw()
#c3.SaveAs(f"output/Delta/dphi_1_{is_sigmalised}{iFile}.png")

In [105]:
c3 = root.TCanvas(f"c2",f"c2",500*2,500*2)
c3.Divide(2,2)
legends =[]
for icentr in range(N_centr):
    c3.cd(icentr+1)
    proj = hists_ep_FG_read[iOption][icentr].ProjectionY(f"px{icentr}{ihist}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj_BG = hists_ep_BG_read[iOption][icentr].ProjectionY(f"px_BG_{icentr}{ihist}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj_BG.Scale(output_array[iOption][icent])
    proj.Rebin(rebin)
    proj_BG.Rebin(rebin)
    Format_Hist_total(proj,"#Delta#phi","dN/d#phi",0.22,0.15,0.03,0.1)
    proj.GetYaxis().SetTitleOffset(1.5)
    proj.Draw("")
    proj.SetMinimum(0)
    proj.SetLineWidth(3)
    proj_BG.SetLineWidth(3)
    proj_BG.SetLineColor(2)
    proj_BG.Draw("same")
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()
        
    

c3.Draw()
#c3.SaveAs(f"output/Delta/dphi_1_{is_sigmalised}{iFile}.png")

In [107]:
c3 = root.TCanvas(f"c2",f"c2",500*2,500*2)
c3.Divide(2,2)
legends =[]
iOption = 0
rebin = 2
for icentr in range(N_centr):
    c3.cd(icentr+1)
    proj = hists_ep_FG_read[iOption][icentr].ProjectionY(f"px{icentr}{ihist}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj_BG = hists_ep_BG_read[iOption][icentr].ProjectionY(f"px_BG_{icentr}{ihist}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj_BG.Scale(output_array[iOption][icent])
    proj.Add(proj_BG,-0.99)
    proj.Rebin(rebin)
    proj_BG.Rebin(rebin)
    Format_Hist_total(proj,"#Delta#phi","dN/d#phi",0.22,0.15,0.03,0.1)
    proj.GetYaxis().SetTitleOffset(1.5)
    proj.Draw("")
    proj.SetMinimum(0)
    proj.SetLineWidth(3)
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()
        
    

c3.Draw()
#c3.SaveAs(f"output/Delta/dphi_1_{is_sigmalised}{iFile}.png")